In [1]:
def perm_equal(a,b):
    '''
    This tests which cyclic permutation of a partition yields equality mod 2
    For odd partitions, there is a unique answer because there is an 
    odd string of zeros between the pairs in exactly one way
    For even partitions WHICH ARE RELELVANT HERE! there could be two ways that the partitions are equal
    because the ones in the partitions could come from distinct odd numbers
    '''
    length = len(a)
    equal = False
    equal_places = []
    for i in range(length):
        b_perm = [b[(j+i)%length] for j in range(length)]
        if a == b_perm:
            equal = True
            equal_places += [i]
    if equal:
        return (True,equal_places)
    else:
        return (False,None)

def extract_cyl_diag(cyl_diag_plus_vdat, part_red):
    length = len(part_red)
    cyl_bot = cyl_diag_plus_vdat[0][0][0]
    cyl_top = cyl_diag_plus_vdat[0][0][1]
    vert_data = cyl_diag_plus_vdat[1]
    align_list = []
    for i in vert_data:
        equal_perm = perm_equal(i,part_red)
        if equal_perm[0]:
            for equal_place in equal_perm[1]:
                align = [cyl_bot[(j+equal_place)%length] for j in range(length)]
                align_list.append([align,cyl_top])
#                print(align, i)
    return align_list

def cyl_diag_plus_one_partition_align(part, master_cyl_diags_plus_data):
    total = []
    bin_list = [j%2 for j in part]
    for cyl_diag_plus_vdat in master_cyl_diags_plus_data:
        proc_diag = extract_cyl_diag(cyl_diag_plus_vdat, bin_list)
        if len(proc_diag) > 0:
            total.append((part,proc_diag))
    return total

def make_align_list_from_part(part_list, master_cyl_diags_plus_data):
    total = []
    print(str(len(part_list)) + ' partitions to search')
    count = 0
    for part in part_list:
        count += 1
        total += cyl_diag_plus_one_partition_align(part, master_cyl_diags_plus_data)
        if count%10000==0:
            print(str(count) + ' partitions processed', part)
    return total

def align_list_write_file(part_list, master_cyl_diags_plus_data, filename_root, t0 = None):
    if t0 == None:
        align_list = make_align_list_from_part(part_list, master_cyl_diags_plus_data)
        filename = filename_root
    else:
        sub_part_list = [part for part in part_list if part[0] == t0]
        align_list = make_align_list_from_part(sub_part_list, master_cyl_diags_plus_data)
        filename = filename_root + '_' + str(t0)
    with open(filename, 'w') as file:
        file.write(str(align_list))
    return filename + ' written'

In [2]:
def eval_align_list(align_list_string, print_status = False):
    #Not clear how much memory this takes, but a 3GB file took 18GB-19GB to open
    #It may have eaten up another 13GB, but other files are running at the same time
    pre_string1 = align_list_string[1:]
    pre_string = pre_string1[:-1]
    if print_status:
        print('pre_string')
    split_part_string_no_paren = re.split(']]]\),\s', pre_string)
#    print(split_part_string_no_paren[-1])
    if print_status:
        print('split_part_string_no_paren')
    split_part_string = [j+']]])' for j in split_part_string_no_paren[:-1]] + [split_part_string_no_paren[-1]]
    if print_status:
        print('split_part_string')
#    split_part_string[-1] = split_part_string[-1]
    if print_status:
        print('last term')
        print(len(split_part_string))
    #Everything up to this point doesn't require too much extra memory
    #To really make this efficient we really have to redo the return statement below
    return [eval(j) for j in split_part_string]